In [2]:
from elasticsearch import Elasticsearch
es = Elasticsearch("http://localhost:9210", timeout=100)

In [59]:
import json
mapping={}
with open('ngram_analyzer.json') as f:
    mapping = json.load(f)
if es.indices.exists(index="ngram_analyzer"):
    es.indices.delete(index="ngram_analyzer")
es.indices.create(index="ngram_analyzer", body=mapping)

{'acknowledged': True}

In [3]:
phrase = "真理があなたがたを自由にする"
# kuromojiを指定して解析テスト
body_ = {"analyzer": "kuromoji_analyzer", "text": phrase}
print(es.indices.analyze(index="ngram_analyzer",body=body_))

{'tokens': [{'token': '真理', 'start_offset': 0, 'end_offset': 2, 'type': 'word', 'position': 0}, {'token': 'が', 'start_offset': 2, 'end_offset': 3, 'type': 'word', 'position': 1}, {'token': 'あなた', 'start_offset': 3, 'end_offset': 6, 'type': 'word', 'position': 2}, {'token': 'がた', 'start_offset': 6, 'end_offset': 8, 'type': 'word', 'position': 3}, {'token': 'を', 'start_offset': 8, 'end_offset': 9, 'type': 'word', 'position': 4}, {'token': '自由', 'start_offset': 9, 'end_offset': 11, 'type': 'word', 'position': 5}, {'token': 'に', 'start_offset': 11, 'end_offset': 12, 'type': 'word', 'position': 6}, {'token': 'する', 'start_offset': 12, 'end_offset': 14, 'type': 'word', 'position': 7}]}


下記の例では、/data/ngramviewer/txtdata/ディレクトリ以下に、資料単位でtarファイルにまとめられた各ページのテキストデータが格納されている形式を想定しています。

スクリプトを実行すると、/data/ngramviewer/ngramdata/ディレクトリ以下に、資料単位のgzip圧縮されたjsonとして、当該資料に出現する1gramから5gramまでのキーワードの頻度情報が、キーバリュー形式で格納されます。


In [5]:
import os
import tarfile
import glob
import gzip
import json
from tqdm import tqdm
INPUTDIR="/data/ngramviewer/txtdata/"
OUTPUTDIR="/data/ngramviewer/ngramdata/"
os.makedirs(OUTPUTDIR,exist_ok=True)

def is_num(s):
    try:
        float(s)
    except ValueError:
        return False
    else:
        return True
for inputtarpath in glob.glob(os.path.join(INPUTDIR,"*.tar")):
    pid=os.path.basename(inputtarpath).split(".")[0]
    resdata={}
    with tarfile.open(inputtarpath, 'r') as tr:
        tarlistinfo=[]
        for tarinfo in tr:
            if "txt" in tarinfo.name:
                tarlistinfo.append(tarinfo.name)
        sorted(tarlistinfo)
        resstr=""
        for tarinfo in tarlistinfo:
            #print(tarinfo)
            binary = tr.extractfile(tarinfo).read()
            tmpstr = binary.decode("utf-8").replace("\t","").replace("\n","")
            resstr+=tmpstr
        body_ = {"analyzer": "kuromoji_analyzer", "text": resstr}
        resdata=es.indices.analyze(index="ngram_analyzer",body=body_,request_timeout=1000)
        resdataarray=[tmp["token"] for tmp in resdata["tokens"]]
    arraylen=len(resdataarray)
    countdict={}
    for pos in range(arraylen):
        tmpstr=""
        for i in range(5):#posの位置を先頭にして5gramまで集計の対象とする
            if pos+i>=arraylen or is_num(resdataarray[pos+i]):
                break
            tmpstr+=(resdataarray[pos+i]+" ")
            if not tmpstr in countdict:
                countdict[tmpstr]=1
            else:
                countdict[tmpstr]+=1
    resjson = json.dumps(countdict,ensure_ascii=False, indent=2)
    with gzip.open(os.path.join(OUTPUTDIR,pid+".json.gz"), mode='wt') as fp:
        fp.write(resjson)